In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
# Path to CSV with image-level predictions
CSV_PATH = "image_feature_predictions.csv"

df = pd.read_csv(CSV_PATH)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'image_feature_predictions.csv'

In [ ]:
IMAGE_COL = "image_id"
FEATURE_COLS = [c for c in df.columns if c != IMAGE_COL]

print("Detected feature columns:")
FEATURE_COLS

In [ ]:
def get_active_features(row, feature_cols):
    """
    Returns a set of features that are 'on' (value == 1) for this image.
    """
    return {feat for feat in feature_cols if row[feat] == 1}


In [ ]:
SINGLE_FEATURE_RULES = {
    "Narrow_Eyes": [
        "Use lighter eyeshadow shades on the inner corners and center of the lid to visually open the eyes.",
        "Focus eyeliner and mascara toward the outer third of the eye to create width."
    ],

    "High_Cheekbones": [
        "Apply blush slightly lower on the cheeks rather than directly on the cheekbones for balance.",
        "Use subtle contour beneath the cheekbones to enhance structure without over-definition."
    ],

    "Rosy_Cheeks": [
        "Use a green-toned color corrector before foundation to neutralize redness.",
        "Choose neutral or peach-toned blushes instead of pink to avoid emphasizing redness."
    ],

    "Pointy_Nose": [
        "Apply soft matte contour along the sides of the nose and blend thoroughly to soften sharp angles.",
        "Avoid strong highlight on the nose tip to reduce emphasis."
    ],

    "Big_Nose": [
        "Use matte contour along the sides of the nose bridge to create balance.",
        "Shift visual focus toward the eyes or lips to draw attention away from the center of the face."
    ],

    "Big_Lips": [
        "Use soft matte or satin lipstick finishes instead of high-gloss to balance lip volume.",
        "Follow the natural lip line and avoid overlining."
    ],

    "Pale_Skin": [
        "Use soft peach or rose blush shades to add warmth to the complexion.",
        "Avoid overly dark contour shades; opt for light, neutral tones instead."
    ],

    "Oval_Face": [
        "Minimal contouring is needed; focus on blush placement to enhance natural balance.",
        "Experiment freely with eye and lip makeup, as most styles complement an oval face."
    ],

    "Arched_Eyebrows": [
        "Follow the natural brow arch and avoid flattening it with heavy filling.",
        "Use lighter brow products and softer strokes to keep the look balanced."
    ]
}
COMBO_RULES = {
    frozenset(["Narrow_Eyes", "Arched_Eyebrows"]): [
        "Keep brow makeup soft while using light-reflecting eyeshadows to open the eye area without over-sharpening features."
    ],

    frozenset(["Narrow_Eyes", "Big_Lips"]): [
        "Use brightening eye makeup to open the eyes while choosing neutral lip tones to maintain facial balance."
    ],

    frozenset(["High_Cheekbones", "Oval_Face"]): [
        "Use blush sparingly and focus on subtle highlighting to enhance structure without overpowering natural balance."
    ],

    frozenset(["Rosy_Cheeks", "Pale_Skin"]): [
        "Prioritize color correction and lightweight foundation to even skin tone while maintaining a natural finish."
    ],

    frozenset(["Pointy_Nose", "Big_Nose"]): [
        "Use soft, blended contouring techniques and avoid harsh highlights to create a more balanced nose appearance."
    ],

    frozenset(["Arched_Eyebrows", "Pointy_Nose"]): [
        "Balance strong facial angles with soft eye makeup and diffused contouring across the center of the face."
    ],

    frozenset(["Big_Lips", "Pale_Skin"]): [
        "Choose muted or cool-toned lip colors and balance with warm blush to avoid high contrast."
    ],

    frozenset(["High_Cheekbones", "Rosy_Cheeks"]): [
        "Apply blush lower on the cheeks and blend outward to reduce emphasis on redness while maintaining structure."
    ]
}


In [ ]:
def generate_recommendations(active_features):
    recs = []

    # 1. Check combo rules
    for combo, combo_recs in COMBO_RULES.items():
        if combo.issubset(active_features):
            recs.extend(combo_recs)

    # 2. Single-feature rules
    for feat in active_features:
        if feat in SINGLE_FEATURE_RULES:
            recs.extend(SINGLE_FEATURE_RULES[feat])

    # 3. Remove duplicates while preserving order
    seen = set()
    final_recs = []
    for r in recs:
        if r not in seen:
            final_recs.append(r)
            seen.add(r)

    return final_recs


In [ ]:
all_results = []

for _, row in df.iterrows():
    image_id = row[IMAGE_COL]
    active_features = get_active_features(row, FEATURE_COLS)
    recommendations = generate_recommendations(active_features)

    all_results.append({
        "image_id": image_id,
        "active_features": list(active_features),
        "recommendations": recommendations
    })

results_df = pd.DataFrame(all_results)
results_df.head()


In [ ]:
def print_recommendations_for_image(image_id):
    row = results_df[results_df["image_id"] == image_id].iloc[0]

    print("Image:", row["image_id"])
    print("Detected features:")
    for f in row["active_features"]:
        print("-", f)

    print("\nRecommendations:")
    for r in row["recommendations"]:
        print("•", r)


In [ ]:
print_recommendations_for_image(results_df.iloc[0]["image_id"])